In [1]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
import pandasql as psql
from pandasql import sqldf

In [2]:
# Define the function to perform T-Tests and calculate Cohen's d
def perform_t_tests(df, metrics, alpha=0.05):
    # Get unique values for 'area' and 'tenured'
    areas = df['area'].unique()
    tenured_statuses = df['tenured'].unique()
    
    # Function to interpret Cohen's d
    def interpret_cohen_d(d):
        if abs(d) < 0.2:
            return "Small"
        elif abs(d) < 0.5:
            return "Medium"
        else:
            return "Large"

    # Initialize a list to store results
    results = []

    for tenured in tenured_statuses:
        for area in areas:
            for metric in metrics:
                # Subset data based on 'area' and 'tenured'
                subset = df[(df['tenured'] == tenured) & (df['area'] == area)]
                # Separate data into 'HOME' and 'OFFICE' groups
                wfh_data = subset[subset['day_type'] == 'HOME'][metric]
                office_data = subset[subset['day_type'] == 'OFFICE'][metric]
                
                if len(wfh_data) > 0 and len(office_data) > 0:
                    # Perform T-Test
                    t_stat, p_value = ttest_ind(wfh_data, office_data, equal_var=False, nan_policy='omit')
                    
                    # Calculate means and sample sizes
                    wfh_mean = wfh_data.mean()
                    office_mean = office_data.mean()
                    wfh_std = wfh_data.std()
                    office_std = office_data.std()
                    n_wfh = len(wfh_data)
                    n_office = len(office_data)
                    
                    # Calculate pooled standard deviation
                    pooled_std = np.sqrt(((n_wfh - 1) * wfh_std**2 + (n_office - 1) * office_std**2) / (n_wfh + n_office - 2))
                    
                    # Calculate Cohen's d
                    cohen_d = (wfh_mean - office_mean) / pooled_std
                    cohen_d_interpretation = interpret_cohen_d(cohen_d)
                    
                    # Append results to the list
                    results.append({
                        'Tenured': tenured,
                        'Area': area,
                        'Metric': metric,
                        'WFH Mean': wfh_mean,
                        'Office Mean': office_mean,
                        'T-Statistic': t_stat,
                        'P-Value': round(p_value, 5),
                        'Cohen D': round(cohen_d, 5),
                        'Cohen D Interpretation': cohen_d_interpretation
                    })
        
        # Add combined area for the current tenured status
        for metric in metrics:
            combined_subset = df[df['tenured'] == tenured]
            wfh_data_combined = combined_subset[combined_subset['day_type'] == 'HOME'][metric]
            office_data_combined = combined_subset[combined_subset['day_type'] == 'OFFICE'][metric]
            
            if len(wfh_data_combined) > 0 and len(office_data_combined) > 0:
                # Perform T-Test
                t_stat_combined, p_value_combined = ttest_ind(wfh_data_combined, office_data_combined, equal_var=False, nan_policy='omit')
                
                # Calculate means and sample sizes
                wfh_mean_combined = wfh_data_combined.mean()
                office_mean_combined = office_data_combined.mean()
                wfh_std_combined = wfh_data_combined.std()
                office_std_combined = office_data_combined.std()
                n_wfh_combined = len(wfh_data_combined)
                n_office_combined = len(office_data_combined)
                
                # Calculate pooled standard deviation
                pooled_std_combined = np.sqrt(((n_wfh_combined - 1) * wfh_std_combined**2 + (n_office_combined - 1) * office_std_combined**2) / (n_wfh_combined + n_office_combined - 2))
                
                # Calculate Cohen's d
                cohen_d_combined = (wfh_mean_combined - office_mean_combined) / pooled_std_combined
                cohen_d_combined_interpretation = interpret_cohen_d(cohen_d_combined)
                
                # Append combined area results to the list
                results.append({
                    'Tenured': tenured,
                    'Area': 'All Areas Combined',
                    'Metric': metric,
                    'WFH Mean': wfh_mean_combined,
                    'Office Mean': office_mean_combined,
                    'T-Statistic': t_stat_combined,
                    'P-Value': round(p_value_combined, 5),
                    'Cohen D': round(cohen_d_combined, 5),
                    'Cohen D Interpretation': cohen_d_combined_interpretation
                })
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Assuming 'result_df' is your DataFrame loaded with your data
metrics = ['avg_interaction_count', 'aht', 'avg_productivity']

# Set the alpha level (e.g., 0.05 for a 95% confidence level)
alpha = 0.05

# Perform the T-Tests and get the results
t_test_results = perform_t_tests(result_df, metrics)

print(t_test_results.head(5))

NameError: name 'result_df' is not defined

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from math import asin, sqrt

# Transcribe the data into a list of dictionaries
data = [
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "Adherence", "WFH Proportion": 0.755},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "Consults", "WFH Proportion": 0.153},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "OSAT", "WFH Proportion": 0.552},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "Transfers", "WFH Proportion": 0.129},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "Adherence", "WFH Proportion": 0.912},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "Consults", "WFH Proportion": 0.206},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "OSAT", "WFH Proportion": 0.472},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "Transfers", "WFH Proportion": 0.148},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "Adherence", "WFH Proportion": 0.836},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "Consults", "WFH Proportion": 0.178},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "OSAT", "WFH Proportion": 0.468},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "Transfers", "WFH Proportion": 0.188},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "Adherence", "WFH Proportion": 0.888},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "Consults", "WFH Proportion": 0.104},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "OSAT", "WFH Proportion": 1.000},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "Transfers", "WFH Proportion": 0.241},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "Adherence", "WFH Proportion": 0.822},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "Consults", "WFH Proportion": 0.266},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "OSAT", "WFH Proportion": 0.571},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "Transfers", "WFH Proportion": 0.273},
]

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Adding some hypothetical Office Proportion data
np.random.seed(42)  # For reproducibility
df["Office Proportion"] = np.random.rand(len(df))

# Set the alpha level (e.g., 0.05 for a 95% confidence level)
alpha = 0.05

# Function to interpret Cohen's h
def interpret_cohen_h(h):
    if abs(h) < 0.2:
        return "Small"
    elif abs(h) < 0.5:
        return "Medium"
    else:
        return "Large"

# Perform Z-tests for each row
results = []

for _, row in df.iterrows():
    count = np.array([row["WFH Proportion"] * row["Sample"], row["Office Proportion"] * row["Sample"]])
    nobs = np.array([row["Sample"], row["Sample"]])
    
    z_score, p_value = proportions_ztest(count, nobs)
    
    # Calculate Cohen's h
    cohen_h = 2 * (asin(sqrt(row["WFH Proportion"])) - asin(sqrt(row["Office Proportion"])))
    
    results.append({
        "Tenured": row["Tenured"],
        "Area Name": row["Area Name"],
        "Sample": row["Sample"],
        "Metric": row["Metric"],
        "WFH Proportion": row["WFH Proportion"],
        "Office Proportion": row["Office Proportion"],
        "Z-Statistic": z_score,
        "P-Value": p_value,
        "Cohen H": cohen_h,
        "Cohen H Interpretation": interpret_cohen_h(cohen_h),
        "Significant": p_value < alpha  # Check if the p-value is less than the alpha level
    })

# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Add combined area for each tenured status
combined_results = []

for tenured in df['Tenured'].unique():
    for metric in df['Metric'].unique():
        combined_subset = df[(df['Tenured'] == tenured) & (df['Metric'] == metric)]
        combined_sample = combined_subset['Sample'].sum()
        combined_wfh_proportion = (combined_subset['WFH Proportion'] * combined_subset['Sample']).sum() / combined_sample
        combined_office_proportion = (combined_subset['Office Proportion'] * combined_subset['Sample']).sum() / combined_sample
        
        count_combined = np.array([combined_wfh_proportion * combined_sample, combined_office_proportion * combined_sample])
        nobs_combined = np.array([combined_sample, combined_sample])
        
        z_score_combined, p_value_combined = proportions_ztest(count_combined, nobs_combined)
        
        # Calculate Cohen's h for combined data
        cohen_h_combined = 2 * (asin(sqrt(combined_wfh_proportion)) - asin(sqrt(combined_office_proportion)))
        
        combined_results.append({
            "Tenured": tenured,
            "Area Name": "COMBINED",
            "Sample": combined_sample,
            "Metric": metric,
            "WFH Proportion": combined_wfh_proportion,
            "Office Proportion": combined_office_proportion,
            "Z-Statistic": z_score_combined,
            "P-Value": p_value_combined,
            "Cohen H": cohen_h_combined,
            "Cohen H Interpretation": interpret_cohen_h(cohen_h_combined),
            "Significant": p_value_combined < alpha  # Check if the p-value is less than the alpha level
        })

# Convert combined results to a DataFrame and append to the original results
df_combined_results = pd.DataFrame(combined_results)
df_final_results = pd.concat([df_results, df_combined_results], ignore_index=True)

# Reorder the columns
df_final_results = df_final_results[["Tenured", "Area Name", "Sample", "Metric", "WFH Proportion", "Office Proportion", "Z-Statistic", "P-Value", "Cohen H Interpretation", "Significant"]]

print(df_final_results.head(5))

In [3]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from math import asin, sqrt

# Define WFH and Office days
wfh_days = ['Tuesday', 'Friday']
office_days = ['Monday', 'Wednesday', 'Thursday']

# Function to interpret Cohen's h
def interpret_cohen_h(h):
    if abs(h) < 0.2:
        return "Small"
    elif abs(h) < 0.5:
        return "Medium"
    else:
        return "Large"

# Function to get the overall difference in proportions by area and tenure
def area_tenure_z_tests(df, metrics_dict):
    area_names = df['area_name'].unique()
    tenured_statuses = df['tenured'].unique()
    results = []

    for tenured in tenured_statuses:
        for area in area_names:
            for metric, (num_col, den_col) in metrics_dict.items():
                subset = df[(df['tenured'] == tenured) & (df['area_name'] == area)]
                wfh_subset = subset[subset['day'].isin(wfh_days)]
                office_subset = subset[subset['day'].isin(office_days)]
                wfh_numerator = wfh_subset[num_col].sum()
                wfh_denominator = wfh_subset[den_col].sum()
                office_numerator = office_subset[num_col].sum()
                office_denominator = office_subset[den_col].sum()

                if wfh_denominator == 0 or office_denominator == 0:
                    continue

                count = np.array([wfh_numerator, office_numerator])
                observations = np.array([wfh_denominator, office_denominator])
                z_stat, p_value = proportions_ztest(count, observations)
                wfh_proportion = wfh_numerator / wfh_denominator if wfh_denominator else None
                office_proportion = office_numerator / office_denominator if office_denominator else None

                # Calculate Cohen's h
                cohen_h = 2 * (asin(sqrt(wfh_proportion)) - asin(sqrt(office_proportion)))
                cohen_h_interpretation = interpret_cohen_h(cohen_h)

                results.append({
                    'Tenured': tenured,
                    'Area Name': area,
                    'Metric': metric,
                    'WFH Proportion': wfh_proportion,
                    'Office Proportion': office_proportion,
                    'Z-Statistic': z_stat,
                    'P-Value': p_value.round(5),
                    'Cohen H': round(cohen_h, 5),
                    'Cohen H Interpretation': cohen_h_interpretation
                })

    return pd.DataFrame(results)

# Function to get the overall difference in proportions by tenure (area agnostic)
def combined_z_tests(df, metrics_dict):
    tenured_statuses = df['tenured'].unique()
    results = []

    for tenured in tenured_statuses:
        for metric, (num_col, den_col) in metrics_dict.items():
            subset = df[df['tenured'] == tenured]
            wfh_subset = subset[subset['day'].isin(wfh_days)]
            office_subset = subset[subset['day'].isin(office_days)]
            wfh_numerator = wfh_subset[num_col].sum()
            wfh_denominator = wfh_subset[den_col].sum()
            office_numerator = office_subset[num_col].sum()
            office_denominator = office_subset[den_col].sum()

            if wfh_denominator == 0 or office_denominator == 0:
                continue

            count = np.array([wfh_numerator, office_numerator])
            observations = np.array([wfh_denominator, office_denominator])
            z_stat, p_value = proportions_ztest(count, observations)
            wfh_proportion = wfh_numerator / wfh_denominator if wfh_denominator else None
            office_proportion = office_numerator / office_denominator if office_denominator else None

            # Calculate Cohen's h
            cohen_h = 2 * (asin(sqrt(wfh_proportion)) - asin(sqrt(office_proportion)))
            cohen_h_interpretation = interpret_cohen_h(cohen_h)

            results.append({
                'Tenured': tenured,
                'Area Name': "COMBINED",
                'Metric': metric,
                'WFH Proportion': wfh_proportion,
                'Office Proportion': office_proportion,
                'Z-Statistic': z_stat,
                'P-Value': p_value.round(5),
                'Cohen H': round(cohen_h, 5),
                'Cohen H Interpretation': cohen_h_interpretation
            })

    return pd.DataFrame(results)

# Test

metrics_dict = {
    'Adherence': ('adh_num', 'adh_den'),
    'OSAT': ('top_box', 'osat_count'),
    'Transfers': ('transfers', 'interaction_count'),
    'Consults': ('consults', 'interaction_count')
}

area_tenure_z_test_df = area_tenure_z_tests(result_df, metrics_dict)
combined_z_test_df = combined_z_tests(result_df, metrics_dict)

z_test = pd.concat([area_tenure_z_test_df, combined_z_test_df])

print(z_test.head(5))

NameError: name 'result_df' is not defined

In [4]:
import pandas as pd

# Data for the DataFrame
data = {
    'day_type': [
        'OFFICE', 'OFFICE', 'OFFICE', 'OFFICE', 'OFFICE',
        'HOME', 'HOME', 'HOME', 'HOME', 'HOME',
        'OFFICE', 'HOME', 'OFFICE', 'HOME', 'OFFICE'
    ],
    'date': [
        '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', 
        '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', 
        '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03'
    ],
    'person_id': [
        'A003083', 'A003091', 'A003434', 'A003687', 'A003771', 
        'A004173', 'A004173', 'A004432', 'A005648', 'A007225', 
        'A008689', 'A009607', 'A009687', 'A015864', 'A017412'
    ],
    'area_name': [
        'High Net Worth', 'Chat', 'High Net Worth', 'Trader Service', 'High Net Worth', 
        'Core Service', 'Core Service', 'Trader Service', 'Trader Service', 'Chat', 
        'High Net Worth', 'Core Service', 'Core Service', 'High Net Worth', 'Core Service'
    ],
    'tenured': ['Yes'] * 15,
    'talk_time': [
        2353, 34945, 6693, 12467, 6444, 
        18897, 18897, 11115, 7036, 13667, 
        3786, 12718, 13221, 2695, 2200
    ],
    'wrap_time': [
        749, 0, 1402, 4207, 84, 
        3399, 3399, 1736, 4361, 0, 
        27, 2010, 663, 27, 749
    ],
    'interaction_count': [
        9, 51, 12, 26, 7, 
        33, 33, 14, 22, 18, 
        10, 15, 18, 9, 4
    ],
    'total_head_ct': [
        2468, 2468, 2468, 2468, 2468, 
        2468, 2468, 2468, 2468, 2468, 
        2468, 2468, 2468, 2468, 2468
    ],
    'tenure_head_ct': [
        1840, 1840, 1840, 1840, 1840, 
        1840, 1840, 1840, 1840, 1840, 
        1840, 1840, 1840, 1840, 1840
    ],
    'area_tenure_head_ct': [
        241, 209, 241, 326, 241, 
        764, 764, 326, 326, 209, 
        241, 764, 764, 241, 764
    ]
}

# Create the DataFrame
result_df = pd.DataFrame(data)

# Display the DataFrame
print(result_df)

   day_type        date person_id       area_name tenured  talk_time  \
0    OFFICE  2024-01-03   A003083  High Net Worth     Yes       2353   
1    OFFICE  2024-01-03   A003091            Chat     Yes      34945   
2    OFFICE  2024-01-03   A003434  High Net Worth     Yes       6693   
3    OFFICE  2024-01-03   A003687  Trader Service     Yes      12467   
4    OFFICE  2024-01-03   A003771  High Net Worth     Yes       6444   
5      HOME  2024-01-03   A004173    Core Service     Yes      18897   
6      HOME  2024-01-03   A004173    Core Service     Yes      18897   
7      HOME  2024-01-03   A004432  Trader Service     Yes      11115   
8      HOME  2024-01-03   A005648  Trader Service     Yes       7036   
9      HOME  2024-01-03   A007225            Chat     Yes      13667   
10   OFFICE  2024-01-03   A008689  High Net Worth     Yes       3786   
11     HOME  2024-01-03   A009607    Core Service     Yes      12718   
12   OFFICE  2024-01-03   A009687    Core Service     Yes      1

In [5]:
query = """
SELECT 
  day_type
, area_name
, tenured
, area_tenure_head_ct AS sample
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4

UNION

SELECT 
  day_type
, "COMBINED" AS area_name
, tenured
, tenure_head_ct AS sample
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4


UNION

SELECT 
  day_type
, "COMBINED" AS area_name
, "COMBINED" AS tenured
, total_head_ct AS sample
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4

"""

aht_df = sqldf(query)
print(aht_df.head(20))

   day_type       area_name   tenured  sample       aht
0      HOME        COMBINED  COMBINED    2468  694.1458
1      HOME        COMBINED       Yes    1840  694.1458
2      HOME            Chat       Yes     209  759.2778
3      HOME    Core Service       Yes     764  732.3457
4      HOME  High Net Worth       Yes     241  302.4444
5      HOME  Trader Service       Yes     326  673.5556
6    OFFICE        COMBINED  COMBINED    2468  656.8613
7    OFFICE        COMBINED       Yes    1840  656.8613
8    OFFICE            Chat       Yes     209  685.1961
9    OFFICE    Core Service       Yes     764  765.1364
10   OFFICE  High Net Worth       Yes     241  566.7895
11   OFFICE  Trader Service       Yes     326  641.3077


In [6]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np

# Function to calculate Cohen's d
def cohen_d(x, y):
    return (np.mean(x) - np.mean(y)) / np.sqrt((np.std(x, ddof=1) ** 2 + np.std(y, ddof=1) ** 2) / 2)

# Function to calculate aht
def calculate_aht(group):
    return (group['talk_time'] + group['wrap_time']).sum() / group['interaction_count'].sum()

# Function to perform the t-test
def ttest_wfh_vs_office(df, area_name, tenured):
    wfh = df[(df['day_type'] == 'HOME') & (df['area_name'] == area_name) & (df['tenured'] == tenured)]
    office = df[(df['day_type'] == 'OFFICE') & (df['area_name'] == area_name) & (df['tenured'] == tenured)]
    
    if len(wfh) > 0 and len(office) > 0:
        wfh_aht = calculate_aht(wfh)
        office_aht = calculate_aht(office)
        
        t_stat, p_value = ttest_ind(wfh_aht, office_aht, equal_var=False)
        effect_size = cohen_d(wfh_aht, office_aht)
        
        return [tenured, area_name, wfh_aht, office_aht, t_stat, p_value, effect_size]
    else:
        return [tenured, area_name, None, None, None, None, None]

# List to hold the results
results = []

# Unique area names and tenured status
area_names = result_df['area_name'].unique()
tenured_statuses = result_df['tenured'].unique()

# Perform t-tests for each area_name and tenured status
for area_name in area_names:
    for tenured in tenured_statuses:
        results.append(ttest_wfh_vs_office(result_df, area_name, tenured))

# Add combination cases
results.append(ttest_wfh_vs_office(result_df, "COMBINED", "Yes"))
results.append(ttest_wfh_vs_office(result_df, "COMBINED", "No"))
results.append(ttest_wfh_vs_office(result_df, "COMBINED", "COMBINED"))

# Convert results to a DataFrame
output_columns = ["Tenured", "Area_Name", "WFH Mean", "Office Mean", "T-Statistic", "P-Value", "Effect Size"]
results_df = pd.DataFrame(results, columns=output_columns)

# Display the results
print(results_df.head(20))

    Tenured       Area_Name    WFH Mean  Office Mean  T-Statistic  P-Value  \
0       Yes  High Net Worth  302.444444   566.789474          NaN      NaN   
1       Yes            Chat  759.277778   685.196078          NaN      NaN   
2       Yes  Trader Service  673.555556   641.307692          NaN      NaN   
3       Yes    Core Service  732.345679   765.136364          NaN      NaN   
4       Yes        COMBINED         NaN          NaN          NaN      NaN   
5        No        COMBINED         NaN          NaN          NaN      NaN   
6  COMBINED        COMBINED         NaN          NaN          NaN      NaN   

   Effect Size  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  
6          NaN  


C:\Users\tshob\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\tshob\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\tshob\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
